# Topological Divergences of Time Series

Data analysis for time series generated by chaotic maps, over a range of control parameter values, using various topological divergences. Classical Lyapunov estimators and recent TDA/HVG measures are also computed as baselines. 

## Set up parallel processing

Ensure cluster is running before executing the code below.

Start a cluster with 32 cores with `ipcluster start -n 32`.

Ensure cluster is stopped after code is complete.

Stop the cluster with `ipcluster stop` in a separate terminal.

In [1]:
import ipyparallel as ipp
clients = ipp.Client()
dv = clients.direct_view()
lbv = clients.load_balanced_view()

## Import modules, classes, and functions

In [2]:
import numpy as np
import networkx as nx

from scipy import stats
from functools import partial

from numpy.random import MT19937
from numpy.random import RandomState
from numpy.random import SeedSequence

from nolds import lyap_r
from nolds import lyap_e

from teaspoon.SP import network_tools
from teaspoon.SP.network import ordinal_partition_graph
from teaspoon.SP.network import knn_graph
from teaspoon.SP.network_tools import remove_zeros
from teaspoon.SP.network_tools import make_network
from teaspoon.SP.tsa_tools import takens
from teaspoon.parameter_selection.MsPE import MsPE_tau
from teaspoon.TDA.Persistence import BettiCurve
from teaspoon.TDA.PHN import DistanceMatrix
from teaspoon.TDA.PHN import PH_network
from teaspoon.TDA.PHN import point_summaries
from gtda.time_series import takens_embedding_optimal_parameters

from LogisticMapLCE import logistic_lce
from HenonMapLCE import henon_lce
from IkedaMapLCE import ikeda_lce
from TinkerbellMapLCE import tinkerbell_lce

from TimeSeriesHVG import TimeSeriesHVG as TSHVG
from TimeSeriesMergeTreeSimple import TimeSeriesMergeTree as TSMT
from TimeSeriesPersistence import TimeSeriesPersistence as TSPH

## Configure the experiment data

In [3]:
# draw samples from a known random state for reproducibility
SEED = 42
randomState = RandomState(MT19937(SeedSequence(SEED)))

TIME_SERIES_LENGTH = 1000
NUM_CONTROL_PARAM_SAMPLES = 100

### Logistic

In [4]:
logistic_control_params = [
    dict(r=r) for r in np.sort(randomState.uniform(3.5, 4.0, NUM_CONTROL_PARAM_SAMPLES))
]
logistic_dataset = [
    logistic_lce(mapParams=params, nIterates=TIME_SERIES_LENGTH, includeTrajectory=True)
    for params in logistic_control_params
]
logistic_trajectories = [
    data["trajectory"][:,0]
    for data in logistic_dataset
]
logistic_lces = np.array([data["lce"][0] for data in logistic_dataset])


### Hénon

In [5]:
henon_control_params = [
    dict(a=a, b=0.3) for a in np.sort(randomState.uniform(0.8, 1.4, NUM_CONTROL_PARAM_SAMPLES))
]
henon_dataset = [
    henon_lce(mapParams=params, nIterates=TIME_SERIES_LENGTH, includeTrajectory=True)
    for params in henon_control_params
]
henon_trajectories = [
    data["trajectory"][:,1]
    for data in henon_dataset
]
henon_lces = np.array([data["lce"][0] for data in henon_dataset])

### Ikeda

In [6]:
ikeda_control_params = [
    dict(a=a) for a in np.sort(randomState.uniform(0.5, 1.0, NUM_CONTROL_PARAM_SAMPLES))
]
ikeda_dataset = [
    ikeda_lce(mapParams=params, nIterates=TIME_SERIES_LENGTH, includeTrajectory=True)
    for params in ikeda_control_params
]
ikeda_trajectories = [
    data["trajectory"][:,0]
    for data in ikeda_dataset
]
ikeda_lces = np.array([data["lce"][0] for data in ikeda_dataset])


### Tinkerbell

In [7]:
tinkerbell_control_params = [
    dict(a=a) for a in np.sort(randomState.uniform(0.7, 0.9, NUM_CONTROL_PARAM_SAMPLES))
]
tinkerbell_dataset = [
    tinkerbell_lce(mapParams=params, nIterates=TIME_SERIES_LENGTH, includeTrajectory=True)
    for params in tinkerbell_control_params
]
tinkerbell_trajectories = [
    data["trajectory"][:,0]
    for data in tinkerbell_dataset
]
tinkerbell_lces = np.array([data["lce"][0] for data in tinkerbell_dataset])


## Build time series representations

In [13]:
def build_representation(dataset, rep_class, rep_class_kwargs):
    trajectories = [data["trajectory"][:,0] for data in dataset]
    return [rep_class(ts, **rep_class_kwargs) for ts in trajectories]

In [9]:
tshvg_kwargs = dict(
    DEGREE_DISTRIBUTION_MAX_DEGREE=100,
    DEGREE_DISTRIBUTION_DIVERGENCE_P_VALUE=1.0,
    directed=None,
    weighted=None,
    penetrable_limit=0,
)

In [10]:
tsmt_kwargs = dict(
    INTERLEAVING_DIVERGENCE_MESH=0.25,
)

In [11]:
tsph_kwargs = dict(
    ENTROPY_SUMMARY_RESOLUTION=100,
    BETTI_CURVE_RESOLUTION=100,
    BETTI_CURVE_NORM_P_VALUE=1.0,
    SILHOUETTE_RESOLUTION=100,
    SILHOUETTE_WEIGHT=1,
    LIFESPAN_CURVE_RESOLUTION=100,
    IMAGE_BANDWIDTH=0.2,
    IMAGE_RESOLUTION=20,
    ENTROPY_SUMMARY_DIVERGENCE_P_VALUE=2.0,
    PERSISTENCE_STATISTICS_DIVERGENCE_P_VALUE=2.0,
    WASSERSTEIN_DIVERGENCE_P_VALUE=1.0,
    BETTI_CURVE_DIVERGENCE_P_VALUE=1.0,
    PERSISTENCE_SILHOUETTE_DIVERGENCE_P_VALUE=2.0,
    PERSISTENCE_LIFESPAN_DIVERGENCE_P_VALUE=2.0,
)

### Logistic

In [14]:
logistic_tshvgs = build_representation(logistic_dataset, TSHVG, tshvg_kwargs)
logistic_tsmts = build_representation(logistic_dataset, TSMT, tsmt_kwargs)
logistic_tsphs = build_representation(logistic_dataset, TSPH, tsph_kwargs)

### Hénon

In [15]:
henon_tshvgs = build_representation(henon_dataset, TSHVG, tshvg_kwargs)
henon_tsmts = build_representation(henon_dataset, TSMT, tsmt_kwargs)
henon_tsphs = build_representation(henon_dataset, TSPH, tsph_kwargs)

### Ikeda

In [16]:
ikeda_tshvgs = build_representation(ikeda_dataset, TSHVG, tshvg_kwargs)
ikeda_tsmts = build_representation(ikeda_dataset, TSMT, tsmt_kwargs)
ikeda_tsphs = build_representation(ikeda_dataset, TSPH, tsph_kwargs)

### Tinkerbell

In [17]:
tinkerbell_tshvgs = build_representation(tinkerbell_dataset, TSHVG, tshvg_kwargs)
tinkerbell_tsmts = build_representation(tinkerbell_dataset, TSMT, tsmt_kwargs)
tinkerbell_tsphs = build_representation(tinkerbell_dataset, TSPH, tsph_kwargs)

## Get Lyapunov exponents and topological divergences

### Lyapunov exponents (ground truth)

Calculated using numerical integration and the Benettin algorithm above.

### Helper functions

In [18]:
def dict_of_arrays(list_of_dicts):
    """Convert list of dictionaries with equal keys to a dictionary of numpy arrays.
    
    Example
        Input
            [{'a': 1, 'b': 2}, {'a': 3, 'b': 4}]
        Output
            {'a': np.array([1, 3]), 'b': np.array([2, 4])}
    """
    return {key: np.array([d[key] for d in list_of_dicts]) for key in list_of_dicts[0]}

def topological_divergences(ts_representations):
    divergences = dv.map_sync(lambda rep: rep.divergences, ts_representations)
    return dict_of_arrays(divergences)

### HVG divergences

Wasserstein and $L_p$ divergences of the time series HVG degree distributions.

In [19]:
logistic_hvg_divergences = topological_divergences(logistic_tshvgs)
henon_hvg_divergences = topological_divergences(henon_tshvgs)
ikeda_hvg_divergences = topological_divergences(ikeda_tshvgs)
tinkerbell_hvg_divergences = topological_divergences(tinkerbell_tshvgs)

### Merge tree divergences

Interleaving divergence and leaf-to-offset-leaf path length distribution divergence.

In [21]:
logistic_mt_divergences = topological_divergences(logistic_tsmts)
henon_mt_divergences = topological_divergences(henon_tsmts)
ikeda_mt_divergences = topological_divergences(ikeda_tsmts)
tinkerbell_mt_divergences = topological_divergences(tinkerbell_tsmts)

In [ ]:
from matplotlib import pyplot as plt
plt.plot(henon_lces, lw=0.7)
plt.show()
plt.plot([div["interleaving"] for div in henon_mt_divergences])
plt.show()

In [ ]:
henon_mt_divergences

### Persistent homology divergences

Various divergences based on the superlevel and sublevel persistence diagrams.

In [ ]:
logistic_ph_divergences = topological_divergences(logistic_tsphs)
henon_ph_divergences = topological_divergences(henon_tsphs)
ikeda_ph_divergences = topological_divergences(ikeda_tsphs)
tinkerbell_ph_divergences = topological_divergences(tinkerbell_tsphs)

In [ ]:
logistic_ph_divergences

## Baselines

Other measures that might approximate or estimate the largest Lyapunov exponent of the trajectory data.

### Classical measures

The Rosenstein and Eckmann estimates from Python `nolds`.

In [ ]:
logistic_rosenstein_estimates = np.array(dv.map_sync(lyap_r, logistic_trajectories))
henon_rosenstein_estimates = np.array(dv.map_sync(lyap_r, henon_trajectories))
ikeda_rosenstein_estimates = np.array(dv.map_sync(lyap_r, ikeda_trajectories))
tinkerbell_rosenstein_estimates = np.array(dv.map_sync(lyap_r, tinkerbell_trajectories))


In [ ]:
logistic_rosenstein_estimates

In [ ]:
logistic_eckmann_estimates = np.array([x[0] for x in dv.map_sync(lyap_e, logistic_trajectories)])
henon_eckmann_estimates = np.array([x[0] for x in dv.map_sync(lyap_e, henon_trajectories)])
ikeda_eckmann_estimates = np.array([x[0] for x in dv.map_sync(lyap_e, ikeda_trajectories)])
tinkerbell_eckmann_estimates = np.array([x[0] for x in dv.map_sync(lyap_e, tinkerbell_trajectories)])


In [ ]:
logistic_eckmann_estimates

### HVG-based measures

The $L_1$ distance between degree distributions of top and bottom HVGs. See Hasson _et. al._ (2018).

This is already computed above as `logistic_hvg_divergences["degree_lp"]`.

### TDA-based measures

1. The point summary statistics of persistent homology of kNN graphs of Takens embeddings of time series. See Myers _et. al._ (2019).
2. The norm of Betti curves of the Vietorix Rips persistence on the full n-dimensional state space trajectory. See Güzel _et. al._ (2022). 

### Helper functions

In [ ]:
def DistanceMatrixFixed(A):
    """Get the all-pairs unweighted shortest path lengths in the graph A.

    Fixes an issue in the `teaspoon` library such that distance matrix computation
    fails with disconnected graphs A.
    """

    A = network_tools.remove_zeros(A)
    np.fill_diagonal(A, 0)
    A = A + A.T

    A_sp = np.copy(A)
    N = len(A_sp)
    D = np.zeros((N,N))

    A_sp[A_sp > 0] = 1
    G = nx.from_numpy_matrix(A_sp)
    lengths = dict(nx.all_pairs_shortest_path_length(G))    
    for i in range(N-1):
        for j in range(i+1, N):
            D[i][j] = lengths.get(i, {}).get(j, np.inf)
    D = D + D.T
    return D


### $k$-NN persistence point summaries

In [ ]:
EMBED_MAX_DIM = 5
EMBED_MAX_DELAY = 40

logistic_optimal_embedding_params = [
    takens_embedding_optimal_parameters(
        ts, max_dimension=EMBED_MAX_DIM, max_time_delay=EMBED_MAX_DELAY
    )
    for ts in logistic_trajectories
]
henon_optimal_embedding_params = [
    takens_embedding_optimal_parameters(
        ts, max_dimension=EMBED_MAX_DIM, max_time_delay=EMBED_MAX_DELAY
    )
    for ts in henon_trajectories
]
ikeda_optimal_embedding_params = [
    takens_embedding_optimal_parameters(
        ts, max_dimension=EMBED_MAX_DIM, max_time_delay=EMBED_MAX_DELAY
    )
    for ts in ikeda_trajectories
]
tinkerbell_optimal_embedding_params = [
    takens_embedding_optimal_parameters(
        ts, max_dimension=EMBED_MAX_DIM, max_time_delay=EMBED_MAX_DELAY
    )
    for ts in tinkerbell_trajectories
]


In [ ]:
logistic_embeddings = [
    takens(ts, n=dim, tau=tau)
    for ts, (tau, dim) in zip(logistic_trajectories, logistic_optimal_embedding_params)
]
henon_embeddings = [
    takens(ts, n=dim, tau=tau)
    for ts, (tau, dim) in zip(henon_trajectories, henon_optimal_embedding_params)
]
ikeda_embeddings = [
    takens(ts, n=dim, tau=tau)
    for ts, (tau, dim) in zip(ikeda_trajectories, ikeda_optimal_embedding_params)
]
tinkerbell_embeddings = [
    takens(ts, n=dim, tau=tau)
    for ts, (tau, dim) in zip(
        tinkerbell_trajectories, tinkerbell_optimal_embedding_params
    )
]


In [ ]:
K_NEIGHBOURS = 4

knn_graph_parallel = partial(knn_graph, k=K_NEIGHBOURS)

logistic_knn_graphs = dv.map_sync(knn_graph_parallel, logistic_trajectories)
henon_knn_graphs = dv.map_sync(knn_graph_parallel, henon_trajectories)
ikeda_knn_graphs = dv.map_sync(knn_graph_parallel, ikeda_trajectories)
tinkerbell_knn_graphs = dv.map_sync(knn_graph_parallel, tinkerbell_trajectories)

In [ ]:
logistic_knn_graphs = list(map(remove_zeros, logistic_knn_graphs))
henon_knn_graphs = list(map(remove_zeros, henon_knn_graphs))
ikeda_knn_graphs = list(map(remove_zeros, ikeda_knn_graphs))
tinkerbell_knn_graphs = list(map(remove_zeros, tinkerbell_knn_graphs))

In [ ]:
def distance_matrix_fixed(A):
    """Get the all-pairs unweighted shortest path lengths in the graph A.

    Fixes an issue in the `teaspoon` library such that distance matrix computation
    fails with disconnected graphs A.
    """

    # include imports so function can run in ipyparallel workers
    import numpy as np
    import networkx as nx
    from teaspoon.SP.network_tools import remove_zeros

    A = remove_zeros(A)
    np.fill_diagonal(A, 0)
    A = A + A.T

    A_sp = np.copy(A)
    N = len(A_sp)
    D = np.zeros((N,N))

    A_sp[A_sp > 0] = 1
    G = nx.from_numpy_matrix(A_sp)
    lengths = dict(nx.all_pairs_shortest_path_length(G))    
    for i in range(N-1):
        for j in range(i+1, N):
            D[i][j] = lengths.get(i, {}).get(j, np.inf)
    D = D + D.T
    return D


In [ ]:
logistic_distance_matrices = dv.map_sync(distance_matrix_fixed, logistic_knn_graphs)
henon_distance_matrices = dv.map_sync(distance_matrix_fixed, henon_knn_graphs)
ikeda_distance_matrices = dv.map_sync(distance_matrix_fixed, ikeda_knn_graphs)
tinkerbell_distance_matrices = dv.map_sync(distance_matrix_fixed, tinkerbell_knn_graphs)

In [ ]:
logistic_knn_diagrams = dv.map_sync(PH_network, logistic_distance_matrices)
henon_knn_diagrams = dv.map_sync(PH_network, henon_distance_matrices)
ikeda_knn_diagrams = dv.map_sync(PH_network, ikeda_distance_matrices)
tinkerbell_knn_diagrams = dv.map_sync(PH_network, tinkerbell_distance_matrices)

In [ ]:
logistic_knn_stats = dv.map_sync(point_summaries, logistic_knn_diagrams, logistic_knn_graphs)
henon_knn_stats = dv.map_sync(point_summaries, henon_knn_diagrams, henon_knn_graphs)
ikeda_knn_stats = dv.map_sync(point_summaries, ikeda_knn_diagrams, ikeda_knn_graphs)
tinkerbell_knn_stats = dv.map_sync(point_summaries, tinkerbell_knn_diagrams, tinkerbell_knn_graphs)

In [ ]:
logistic_knn_stats